Any data source that grows over time is called data stream

eg: new files loading in cloud storage, pub/sub in messaging feed, updates to database captured in a CDC feed

Spark streaming guarantees fault tolerance using checkpoint concept

In [0]:
%run ./Copy-Datasets

In [0]:
%fs ls 'dbfs:/mnt/demo-datasets/bookstore/'

path,name,size,modificationTime
dbfs:/mnt/demo-datasets/bookstore/books-cdc/,books-cdc/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-csv/,books-csv/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-csv-new/,books-csv-new/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-streaming/,books-streaming/,0,0
dbfs:/mnt/demo-datasets/bookstore/customers-json/,customers-json/,0,0
dbfs:/mnt/demo-datasets/bookstore/customers-json-new/,customers-json-new/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders/,orders/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-json-raw/,orders-json-raw/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-json-streaming/,orders-json-streaming/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-new/,orders-new/,0,0


In [0]:
%sql
create or replace temp view books_csv
using csv
options(header=True,delimiter=';',path='dbfs:/mnt/demo-datasets/bookstore/books-csv/')

In [0]:
%sql
create  or replace table books
as select distinct * from books_csv

num_affected_rows,num_inserted_rows


In [0]:
spark.readStream.table("books").createOrReplaceTempView("books_streaming_tmp_vw")

In [0]:
%sql
select * from books_streaming_tmp_vw

book_id,title,author,category,price
B09,Advanced Data Structures,Peter Brass,Computer Science,24
B08,Quantum Computing for Everyone,Chris Bernhardt,Computer Science,41
B07,The Hundred-Page Machine Learning,Andriy Burkov,Computer Science,33
B10,Beginning Database Design Solutions,Rod Stephens,Computer Science,44
B12,Big Data in Practice,Bernard Marr,Computer Science,30
B11,Business Intelligence for Dummies,Swain Scheps,Computer Science,38
B01,The Soul of a New Machine,Tracy Kidder,Computer Science,49
B03,Make Your Own Neural Network,Tariq Rashid,Computer Science,35
B02,Learning JavaScript Design Patterns,Addy Osmani,Computer Science,28
B05,Fluent Python,Luciano Ramalho,Computer Science,47


In [0]:
%sql
select author,count(book_id) total_books from books_streaming_tmp_vw group by 1

author,total_books
Mark W. Spong,1
Chris Bernhardt,1
Tariq Rashid,1
Peter Brass,1
Luciano Ramalho,1
Addy Osmani,1
Andriy Burkov,1
Tracy Kidder,1
Swain Scheps,1
François Chollet,1


In [0]:
%sql
create or replace temp view author_cnts_temp_vw
as select author,count(book_id) total_books from books_streaming_tmp_vw group by 1;

In [0]:
spark.table("author_cnts_temp_vw").writeStream.trigger(processingTime='4 seconds').outputMode("complete").option("checkpointLocation","dbfs:/mnt/demo/checkpoints/").table("author_counts")

Out[26]: <pyspark.sql.streaming.query.StreamingQuery at 0x7ff6d7420ca0>

In [0]:
%sql
select * from author_counts

author,total_books
François Chollet,1
Chris Bernhardt,1
Luciano Ramalho,1
Mark W. Spong,1
Andriy Burkov,1
Tariq Rashid,1
Tracy Kidder,1
Swain Scheps,1
Rod Stephens,1
Bernard Marr,1


In [0]:
%sql
INSERT INTO books
values ("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
        ("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
        ("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35)
        

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from author_counts

author,total_books
François Chollet,1
Chris Bernhardt,2
Luciano Ramalho,1
Mark W. Spong,3
Andriy Burkov,1
Tariq Rashid,1
Tracy Kidder,1
Swain Scheps,1
Rod Stephens,1
Bernard Marr,1


In [0]:
%fs head dbfs:/mnt/demo/checkpoints/offsets/0

v1
{"batchWatermarkMs":0,"batchTimestampMs":1680113042716,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.stateStore.compression.codec":"lz4","spark.sql.streaming.stateStore.rocksdb.formatVersion":"5","spark.sql.streaming.statefulOperator.useStrictDistribution":"true","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"200"}}
{"sourceVersion":1,"reservoirId":"9b6dd4f3-da43-4f78-8668-94ab5c00013a","reservoirVersion":3,"index":0,"isStartingVersion":true}

In [0]:
%sql
INSERT INTO books
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
        ("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30),
        ("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35)

num_affected_rows,num_inserted_rows
3,3


In [0]:
spark.table("author_cnts_temp_vw").writeStream.trigger(availableNow=True).outputMode("Complete").option("checkpointLocation","dbfs:/mnt/demo/checkpoints/").table("author_counts").awaitTermination()

In [0]:
%sql
select * from author_counts

author,total_books
Sudharsan Ravichandiran,2
François Chollet,1
Chris Bernhardt,2
Luciano Ramalho,1
Mark W. Spong,3
Richard Lyons,2
Andriy Burkov,1
Yoav Goldberg,2
Tariq Rashid,1
Tracy Kidder,1


In [0]:
%sql 
select author,count(*) from books group by 1

author,count(1)
Mark W. Spong,3
Chris Bernhardt,2
Tariq Rashid,1
Peter Brass,1
Luciano Ramalho,1
Addy Osmani,1
Andriy Burkov,1
Tracy Kidder,1
Swain Scheps,1
François Chollet,1
